In [2]:
%matplotlib inline
import numpy as np
import numpy.ma as ma
import pandas as pd
from pandas import DataFrame, Series
import matplotlib.pyplot as plt
import seaborn as sns
import timeit
import line_profiler

import datetime

from io import StringIO
from pandas.api.types import CategoricalDtype

import pandas._testing as tm
import re

In [5]:
import re
p = re.compile('ab*')
p # re.compile(r'ab*', re.UNICODE)

re.compile(r'ab*', re.UNICODE)

In [6]:
re.UNICODE.value

32

In [29]:
p = re.compile('[a-t]+')
m = p.match("temupau")
print(m) # <re.Match object; span=(0, 3), match='tem'>

<re.Match object; span=(0, 3), match='tem'>


In [32]:
m.group() # 'tem'
m.start() # 0
m.end()   # 3
m.span()  # (0, 3)

(0, 3)

In [39]:
print(p.match('::: message')) # None
m = p.search('::: message'); print(m) #<re.Match object; span=(4, 11), match='message'>
m.group() #'message'
m.start() # 4
m.end() # 11
m.span() # (4, 11)

None
<re.Match object; span=(4, 11), match='message'>


(4, 11)

In [44]:
p = re.compile(r'[a-r]+')
m = p.match('ramesh tau')
if m:
    print('Matched', m.group())
else:
    print('No Match')
## Output - Matched rame

Matched rame


In [45]:
p = re.compile(r'\d+')
p.findall('12 drummers drumming, 11 pipers piping, 10 lords a-leaping')

['12', '11', '10']

In [49]:
p = re.compile(r'\d+')
iterator = p.finditer('12 drummers drumming, 11 pipers piping, 10 lords a-leaping')
for match in iterator:
    print(match.group())
    print(match.start())
    print(match.end())
    print(match.span())

12
0
2
(0, 2)
11
22
24
(22, 24)
10
40
42
(40, 42)


In [50]:
p = re.compile(r'[a-m]')
p.search('allubro')

<re.Match object; span=(0, 1), match='a'>